In [183]:
import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import datetime
import os

In [184]:
from dbconn import querydb, querydbtopandas

In [185]:
prestadores = querydbtopandas("""select id_pre_prestador,  desc_pre_nombre, Q.deno  
                    from dba.d_pre_prestador a12
                    LEFT JOIN dba.prestad_costo_centros E
                    ON a12.id_pre_prestador = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
                    LEFT JOIN dba.costo_centros Q
                    ON E.centro = Q.centro""")

In [186]:
len(prestadores)

314479

In [187]:
# 96 mil no tienen especialidad
len(prestadores[prestadores["deno"].isna()])

96307

In [188]:
prestadores = prestadores[prestadores["deno"].notna()]

In [189]:
prestadores.deno.nunique()

123

In [190]:
def calculo_scores (fecha, prestadores_):
    tabla_scores = pd.DataFrame(columns = ["prestador", "score", "mes"])

    calificaciones = pd.read_parquet(f'datos_calificaciones/{fecha}', columns = ["Prestador", "Promedio total"]) 
    calificaciones["Promedio total"] = 1 - (calificaciones["Promedio total"] * 0.2)

    cirugias = pd.read_parquet(f'datos_cirugias/{fecha}',columns = ["Prestador", "% en Propias"]) 
    cirugias["% no Propias"] = 1 - cirugias["% en Propias"]
    cirugias.drop("% en Propias",axis=1, inplace=True)

    consultas = pd.read_parquet(f'datos_consultas/{fecha}',columns = ["Prestador", "Participación %"]) 
    consultas["Participación %"] = consultas["Participación %"]/ 100

    desconocimiento = pd.read_parquet(f'datos_desconocimiento/{fecha}',columns = ["Prestador","Q afiliados"]) 

    farmacia = pd.read_parquet(f'datos_farmacia/{fecha}', columns = ["N° Prestador","desvios de la media de su sp"])
    farmacia.rename(columns={"N° Prestador": "Prestador"},inplace=True)

    imagenes =  pd.DataFrame(pd.read_parquet(f'datos_imagenes/{fecha}', columns = ["N° Prescriptor", "Normalidad"]))
    imagenes.rename(columns={"N° Prescriptor": "Prestador"},inplace=True)

    quejas = pd.DataFrame(pd.read_parquet(f'datos_quejas/{fecha}', columns = ["Prestador", "Total de quejas prestador"]))

    metricas = [calificaciones, cirugias, consultas, farmacia, imagenes, desconocimiento, quejas]
    ponderacion = pd.DataFrame(index = ["calificaciones", "cirugias", "consultas", "farmacia", "imagenes", "desconocimiento", "quejas"],
             data = [2,5,5,3,3,4,3], columns = ["ponderacion"])

    dict_metricas = {"calificaciones": calificaciones,
                        "cirugias": cirugias ,
                        "consultas":consultas ,
                        "farmacia":farmacia ,
                        "imagenes": imagenes ,
                        "desconocimiento": desconocimiento ,
                        "quejas": quejas}

    for prestador in prestadores_: 
        mask = []
        for metrica in metricas: 
            mask.append(prestador in metrica.Prestador.values)
        tabla_prestador = ponderacion[mask]
        values = []
        for metrica in tabla_prestador.index:
            values.append(dict_metricas[metrica][dict_metricas[metrica]["Prestador"] == prestador].values[0][1])
        tabla_prestador["scores"] = values

        #para desconocimiento
        if "desconocimiento" in tabla_prestador.index:
            if tabla_prestador.loc["desconocimiento", "scores"]  <= 2:
                tabla_prestador.loc["desconocimiento", "scores"] = 0
            elif (tabla_prestador.loc["desconocimiento", "scores"] > 2) & (tabla_prestador.loc["desconocimiento", "scores"] <= 6) :
                tabla_prestador.loc["desconocimiento", "scores"] = 0.5
            elif tabla_prestador.loc["desconocimiento", "scores"]  > 6:
                tabla_prestador.loc["desconocimiento", "scores"] = 1

        #para farmacia
        if "farmacia" in tabla_prestador.index:
            if tabla_prestador.loc["farmacia", "scores"]  <= 1:
                tabla_prestador.loc["farmacia", "scores"]= 0
            elif (tabla_prestador.loc["farmacia", "scores"] > 1) & (tabla_prestador.loc["farmacia", "scores"] <= 2) :
                tabla_prestador.loc["farmacia", "scores"]= 0.5
            elif tabla_prestador.loc["farmacia", "scores"]  > 2:
                tabla_prestador.loc["farmacia", "scores"]= 1

        # para quejas
        if "quejas" in tabla_prestador.index:
            if tabla_prestador.loc["quejas", "scores"] <= 1:
                tabla_prestador.loc["quejas", "scores"]= 0
            elif tabla_prestador.loc["quejas", "scores"]  == 2:
                tabla_prestador.loc["quejas", "scores"]= 0.5
            elif tabla_prestador.loc["quejas", "scores"]  > 2:
                tabla_prestador.loc["quejas", "scores"] = 1

        if len(tabla_prestador) == 0:
            score = np.nan 
        else:

            if "desconocimiento" not in tabla_prestador.index:
                tabla_prestador = tabla_prestador.append(pd.DataFrame({"ponderacion": [4], "scores":[0]}, index = ["desconocimiento"]))
            if "quejas" not in tabla_prestador.index:
                tabla_prestador = tabla_prestador.append(pd.DataFrame({"ponderacion": [3],"scores":[0]}, index = ["quejas"]))

            tabla_prestador["score_pond"] = (tabla_prestador["ponderacion"] * 100) /tabla_prestador.ponderacion.sum()

            tabla_prestador["score_prestador"]  =  tabla_prestador["scores"] * tabla_prestador["score_pond"]
            score = tabla_prestador["score_pond"].sum() - tabla_prestador["score_prestador"].sum() 

        tabla_scores = tabla_scores.append({"prestador": prestador, "score": score, "mes": fecha[:7]}, ignore_index= True)  
    return tabla_scores

In [191]:
prestadores_ = prestadores.id_pre_prestador.tolist()
fecha = '2019-10.gzip'

In [193]:
scores = calculo_scores(fecha, prestadores_)

In [197]:
scores = scores.dropna()

In [199]:
scores.to_parquet("scores/2019-10.gzip", compression = "gzip")

In [9]:
fechas = [i for i in os.listdir('datos_consultas/') if len(i) == 12]
fechas.sort()

In [14]:
#me quedo solo con ciertos meses que son en los que interseccionan las metricas, que se caclulan con 3 meses de atraso y las que no 
fechas = fechas[2:]

In [119]:
for fecha in fechas[:-1]:
    scores = calculo_scores(fecha, prestadores.id_pre_prestador.tolist()[:100])
    scores = scores.dropna()
    #scores.to_parquet(f'scores/{fecha}',compression='gzip')  

In [16]:
import seaborn as sns
sns.heatmap(scores.isna())

NameError: name 'scores' is not defined

In [20]:
scores.isna().sum()

prestador         0
score        160327
mes               0
dtype: int64

In [22]:
len(prestadores.id_pre_prestador.tolist())

218137